## Code to Chapter 5 of LangChain for Life Science and Healthcare book, by Dr. Ivan Reznikov

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1wln-l-rvC_CM_OZ11psJIU_bJba6ETeh?usp=sharing)

This notebook demonstrates how to build personal assistants using LangChain's chain capabilities. We'll explore various chain types including history-aware retrievers, sequential chains, and parallel chains for different use cases in life sciences and research.

## Key Takeaways

- **Chain Composition**: Use LCEL for clean, readable chain building
- **Context Awareness**: History-aware retrievers improve conversational AI
- **Parallel Processing**: Optimize performance with simultaneous operations
- **Modularity**: Break complex tasks into smaller, manageable chains
- **Testing**: Always test chains with various inputs to ensure robustness
- **Visualization**: Use `get_graph().print_ascii()` to understand chain structure

## Package Installation

In [1]:
!pip install -q -U langchain  \
  langchainhub \
  langchain-community \
  langchain-core \
  langchain-experimental \
  langchain-openai \
  langchain-text-splitters \
  langcodes \
  langgraph \
  langsmith \
  libclang \
  openai pandas matplotlib docarray grandalf semanticscholar arxiv xmltodict faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.8/367.8 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 kB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 107.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 103.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.8 MB/s eta 0:00:00
   

In [2]:
!pip freeze | grep "lang\|openai\|chroma"

google-ai-generativelanguage==0.6.15
google-cloud-language==2.17.2
langchain==0.3.26
langchain-community==0.3.27
langchain-core==0.3.68
langchain-experimental==0.3.4
langchain-openai==0.3.28
langchain-text-splitters==0.3.8
langchainhub==0.1.21
langcodes==3.5.0
langgraph==0.5.3
langgraph-checkpoint==2.1.0
langgraph-prebuilt==0.5.2
langgraph-sdk==0.1.73
langsmith==0.4.5
language_data==1.3.0
libclang==18.1.1
openai==1.95.1


In [3]:
from google.colab import userdata
import os


# Set OpenAI API key from Google Colab's user environment or default
def set_api_keys(
    default_openai_key: str = "YOUR_API_KEY", default_tavily_key: str = "YOUR_API_KEY"
) -> None:
    """Set the OpenAI API key from Google Colab's user environment or use a default value."""

    os.environ["OPENAI_API_KEY"] = (
        userdata.get("LC4LS_OPENAI_API_KEY") or default_openai_key
    )


set_api_keys()
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "lc4ls-ch5-lcel-chains"
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")

## History-Aware Retrieval Chain

### What is a History-Aware Retrieval Chain?
A history-aware retrieval chain is a sophisticated RAG (Retrieval-Augmented Generation) system that considers conversation history when retrieving relevant documents. This allows the system to understand context from previous messages and provide more accurate responses.


In [4]:
from langchain import hub
from langchain_core.prompts import MessagesPlaceholder, ChatPromptTemplate
from langchain_core.messages import HumanMessage, AIMessage
from langchain_community.vectorstores import FAISS
from langchain.chains import create_history_aware_retriever
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.documents import Document
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

# Initialize LLM and embedding model
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

# Sample documents (replace with your actual data)
documents = [
    Document(
        page_content="Experiment A: We tested the effect of fertilizer X on tomato yield at temperature 80°F. Results showed a 20% increase in yield in the experimental group."
    ),
    Document(
        page_content="Experiment B: We tested the effect of fertilizer X on corn yield. The experiment is ongoing."
    ),
    Document(
        page_content="Experiment C: We tested the effect of compost and different temperatures on strawberry growth. Best results achieved at 25°C-30°C range."
    ),
    Document(
        page_content="Experiment D: Examined the small influence of compost amount on soil acidity."
    ),
    Document(
        page_content="Experiment E: Examined the influence of light intensity on photosynthesis in algae."
    ),
]

# Create vector store
vector = FAISS.from_documents(documents, embedding_model)
retriever = vector.as_retriever(k=3)

# Prompt for history-aware retrieval
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You help users find information about agricultural experiments.
                Paraphrase the user's query based on the conversation history.""",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
    ]
)

# Create history-aware retriever chain
history_aware_retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

### Test 1: Query with Context Updates

In [5]:
chat_history = [
    HumanMessage(content="Experiment E actually used Fertilizer X."),
    AIMessage(
        content="Thank you for the correction. I'll note that Experiment E used Fertilizer X."
    ),
    HumanMessage(
        content="Experiment B had contamination issues and should be excluded from analysis."
    ),
    AIMessage(
        content="Noted. I'll exclude Experiment B from all analysis due to contamination issues."
    ),
]

In [6]:
query = "List all experiment where fertilizer X was used"

# Invoke the retriever chain
result = history_aware_retriever_chain.invoke(
    {"chat_history": chat_history, "input": query}
)


**Expected Result**: The system should retrieve documents related to fertilizer X, considering the conversation history that Experiment E used Fertilizer X and Experiment B should be excluded.


In [7]:
result

[Document(id='f5c83a87-3a2a-4505-b59b-02a726a4b268', metadata={}, page_content='Experiment B: We tested the effect of fertilizer X on corn yield. The experiment is ongoing.'),
 Document(id='7ccc664d-bb0a-4245-8439-3a809ea5a906', metadata={}, page_content='Experiment A: We tested the effect of fertilizer X on tomato yield at temperature 80°F. Results showed a 20% increase in yield in the experimental group.'),
 Document(id='63618ca3-de82-4633-aa37-ee5111bd5388', metadata={}, page_content='Experiment D: Examined the small influence of compost amount on soil acidity.'),
 Document(id='031c5d80-ac69-4e33-8825-9794f590131a', metadata={}, page_content='Experiment E: Examined the influence of light intensity on photosynthesis in algae.')]

In [8]:
system_prompt = """You are an assistant for question-answering tasks.
    Use the following pieces of retrieved context to answer
    the question. If you don't know the answer, say that you
    don't know. Always use the metric system to answer questions!
    {context}"""

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever_chain, question_answer_chain)

In [9]:
result = rag_chain.invoke({"input": query, "chat_history": chat_history})

In [10]:
result

{'input': 'List all experiment where fertilizer X was used',
 'chat_history': [HumanMessage(content='Experiment E actually used Fertilizer X.', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Thank you for the correction. I'll note that Experiment E used Fertilizer X.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Experiment B had contamination issues and should be excluded from analysis.', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Noted. I'll exclude Experiment B from all analysis due to contamination issues.", additional_kwargs={}, response_metadata={})],
 'context': [Document(id='f5c83a87-3a2a-4505-b59b-02a726a4b268', metadata={}, page_content='Experiment B: We tested the effect of fertilizer X on corn yield. The experiment is ongoing.'),
  Document(id='7ccc664d-bb0a-4245-8439-3a809ea5a906', metadata={}, page_content='Experiment A: We tested the effect of fertilizer X on tomato yield at temperature 80°F. Results showe

### Test 2: Query Without Initial Context

In [11]:
query = "List all experiment where Fertilizer Y was used"

# Invoke the retriever chain
result = history_aware_retriever_chain.invoke({"chat_history": [], "input": query})

**Expected Result**: The system should not find any experiments with Fertilizer Y since none are mentioned in the documents.

In [12]:
result

[Document(id='f5c83a87-3a2a-4505-b59b-02a726a4b268', metadata={}, page_content='Experiment B: We tested the effect of fertilizer X on corn yield. The experiment is ongoing.'),
 Document(id='7ccc664d-bb0a-4245-8439-3a809ea5a906', metadata={}, page_content='Experiment A: We tested the effect of fertilizer X on tomato yield at temperature 80°F. Results showed a 20% increase in yield in the experimental group.'),
 Document(id='63618ca3-de82-4633-aa37-ee5111bd5388', metadata={}, page_content='Experiment D: Examined the small influence of compost amount on soil acidity.'),
 Document(id='031c5d80-ac69-4e33-8825-9794f590131a', metadata={}, page_content='Experiment E: Examined the influence of light intensity on photosynthesis in algae.')]

### Test 3: Query with Semantic Understanding

In [13]:
chat_history = [
    HumanMessage(content="Was Fertilizer Y used in the experiemnts.?"),
    AIMessage(content="No, Fertilizer Y wasn't used in any of the experiments."),
    HumanMessage(content="That's incorrect. Fertilizer Y and compost are the same."),
    AIMessage(
        content="Thank you for the correction. I'll note that Fertilizer Y and compost are the same."
    ),
]

In [14]:
query = "List all experiment where Fertilizer Y was used"

# Create history-aware retriever chain
history_aware_retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

# Invoke the retriever chain
result = history_aware_retriever_chain.invoke(
    {"chat_history": chat_history, "input": query}
)

In [15]:
result

[Document(id='f5c83a87-3a2a-4505-b59b-02a726a4b268', metadata={}, page_content='Experiment B: We tested the effect of fertilizer X on corn yield. The experiment is ongoing.'),
 Document(id='7ccc664d-bb0a-4245-8439-3a809ea5a906', metadata={}, page_content='Experiment A: We tested the effect of fertilizer X on tomato yield at temperature 80°F. Results showed a 20% increase in yield in the experimental group.'),
 Document(id='63618ca3-de82-4633-aa37-ee5111bd5388', metadata={}, page_content='Experiment D: Examined the small influence of compost amount on soil acidity.'),
 Document(id='c7f675d1-596e-4284-98ac-2119abbf67a5', metadata={}, page_content='Experiment C: We tested the effect of compost and different temperatures on strawberry growth. Best results achieved at 25°C-30°C range.')]

In [16]:
system_prompt = """You are an assistant for question-answering tasks.
    Use the following pieces of retrieved context to answer
    the question. If you don't know the answer, say that you
    don't know. Always use the metric system to answer questions!
    {context}"""

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever_chain, question_answer_chain)

In [17]:
result = rag_chain.invoke({"input": query, "chat_history": chat_history})


**Expected Result**: The system should now understand that Fertilizer Y = compost and retrieve experiments C and D that used compost.


In [18]:
result

{'input': 'List all experiment where Fertilizer Y was used',
 'chat_history': [HumanMessage(content='Was Fertilizer Y used in the experiemnts.?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="No, Fertilizer Y wasn't used in any of the experiments.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="That's incorrect. Fertilizer Y and compost are the same.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="Thank you for the correction. I'll note that Fertilizer Y and compost are the same.", additional_kwargs={}, response_metadata={})],
 'context': [Document(id='f5c83a87-3a2a-4505-b59b-02a726a4b268', metadata={}, page_content='Experiment B: We tested the effect of fertilizer X on corn yield. The experiment is ongoing.'),
  Document(id='7ccc664d-bb0a-4245-8439-3a809ea5a906', metadata={}, page_content='Experiment A: We tested the effect of fertilizer X on tomato yield at temperature 80°F. Results showed a 20% increase in yield in the e

## Basic Chain Building with LCEL

### What is LCEL?
LangChain Expression Language (LCEL) is a declarative way to compose chains. It provides a simple syntax for building complex workflows using the pipe operator (|).


In [19]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [20]:
model = ChatOpenAI(model="gpt-3.5-turbo")
prompt = ChatPromptTemplate.from_template(
    "You are a chemist. Answer the following question: {question}"
)
output_parser = StrOutputParser()

### Chain 1: Direct Model Invocation

In [21]:
chain_1 = model
chain_1.invoke(("human", "What is a bond?"))

AIMessage(content="A bond is a fixed income security that represents a loan made by an investor to a borrower, typically a corporation or government entity. When an investor purchases a bond, they are essentially lending money to the issuer in exchange for periodic interest payments and the return of the bond's face value at maturity. Bonds are typically used by companies and governments to raise funds for various projects and expenses.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 76, 'prompt_tokens': 17, 'total_tokens': 93, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BtPSsJz8MgcmEkCf0e49TeHV7mGgu', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--b95402a1-f546-48af-9e

### Chain 2: Prompt + Model

In [22]:
chain_2 = prompt | model
chain_2.invoke({"question": "What is a bond?"})

AIMessage(content='In chemistry, a bond is a force that holds two or more atoms together to form a molecule. Bonds are formed when atoms share or transfer electrons in order to achieve a more stable electron configuration. There are different types of chemical bonds, including covalent bonds, ionic bonds, and metallic bonds. Bonds are essential for the formation of compounds and play a crucial role in determining the properties of substances.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 81, 'prompt_tokens': 23, 'total_tokens': 104, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BtPSuQkpdNTR17vdJ6pIy83ejk5Fy', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--1c2423ba-fa49-

### Chain 3: Complete Chain with Output Parser

In [23]:
chain_3 = prompt | model | output_parser
chain_3.invoke({"question": "What is a bond?"})

'A bond in chemistry refers to the attractive force that holds atoms together in a molecule. This force can be due to the sharing of electrons between atoms (covalent bond), the transfer of electrons from one atom to another (ionic bond), or the attraction between positively and negatively charged ions (ionic bond). Bonds are essential for forming the structure of molecules and determining their physical and chemical properties.'


**Key Difference**: Chain 3 returns a clean string instead of a message object, making it easier to work with programmatically.


## RAG Chain with Parallel Processing

### What is Parallel Processing in Chains?
Parallel processing allows multiple operations to run simultaneously, improving efficiency. In RAG systems, we can retrieve context and pass through the question in parallel.


### Setup Biology Knowledge Base

In [24]:
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI

# Initialize the model
model = ChatOpenAI(model="gpt-3.5-turbo")

vectorstore = DocArrayInMemorySearch.from_texts(
    [
        "DNA carries genetic information within cell chromosomes.",
        "Ecosystems consist of living organisms and their physical environment.",
        "The human heart pumps blood through arteries and veins.",
        "Some bacteria cause diseases while others are beneficial.",
        "Homeostasis maintains steady internal conditions in living systems.",
        "Natural selection helps organisms adapt and survive in their environments.",
        "Mitochondria produce ATP, the main energy source for cells.",
        "Photosynthesis in plants produces oxygen.",
        "The brain controls body functions and is located in the skull.",
        "The immune system defends against harmful substances by detecting antigens.",
    ],
    embedding=OpenAIEmbeddings(model="text-embedding-3-large"),
)
retriever = vectorstore.as_retriever()

/usr/local/lib/python3.11/dist-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


### Create RAG Chain with Parallel Processing

In [25]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

# Setup retrieval and handling of inputs
retrieval = RunnableParallel({"context": retriever, "question": RunnablePassthrough()})

# Combine the components into a processing chain
chain = retrieval | prompt | model | output_parser

### Test Biology RAG Chain

In [26]:
result1 = chain.invoke("How do plants release oxygen?")
print(result1)

Plants release oxygen through photosynthesis.


In [27]:
result2 = chain.invoke("What is the main energy source for cells?")
print(result2)

The main energy source for cells is ATP, which is produced by mitochondria.


In [28]:
result3 = chain.invoke("Who painted mona Lisa?")
print(result3)

The information provided does not mention anything about who painted the Mona Lisa.


**Expected Results**:
- Test 1: Should reference photosynthesis
- Test 2: Should reference ATP and mitochondria
- Test 3: Should indicate the answer is not in the context

## Sequential Chains for Medical Diagnosis

### What are Sequential Chains?
Sequential chains process information in steps, where the output of one chain becomes the input for the next. This is useful for multi-step reasoning tasks.


### Medical Diagnosis Chain

In [29]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# Prompt to determine the disease based on symptoms
prompt_symptom = ChatPromptTemplate.from_template(
    "Based on these symptoms, what disease might this person have: {symptoms}?"
)

# Prompt to recommend lab exams based on the suspected disease
prompt_disease = ChatPromptTemplate.from_template(
    "What lab exams should be taken to confirm a diagnosis of {disease}?"
)

model = ChatOpenAI()

# Chains to determine the disease from symptoms
sub_chain_symptom = prompt_symptom | model | StrOutputParser()
sub_chain_disease = prompt_disease | model | StrOutputParser()

# Sequential chain: symptoms -> disease -> lab tests
main_chain = {"disease": sub_chain_symptom} | sub_chain_disease

### Visualize Chain Structure

In [30]:
main_chain.get_graph().print_ascii()

+------------------------+ 
| Parallel<disease>Input | 
+------------------------+ 
             *             
             *             
             *             
  +--------------------+   
  | ChatPromptTemplate |   
  +--------------------+   
             *             
             *             
             *             
      +------------+       
      | ChatOpenAI |       
      +------------+       
             *             
             *             
             *             
    +-----------------+    
    | StrOutputParser |    
    +-----------------+    
             *             
             *             
             *             
  +--------------------+   
  | ChatPromptTemplate |   
  +--------------------+   
             *             
             *             
             *             
      +------------+       
      | ChatOpenAI |       
      +------------+       
             *             
             *             
             *      

### Test Medical Diagnosis Chain

In [31]:
# Example invocation to test the chain
result = main_chain.invoke({"symptoms": "fever, cough, and shortness of breath"})
print("Recommended Lab Exams:", result)

Recommended Lab Exams: To confirm a diagnosis of pneumonia, influenza, or COVID-19, the following lab tests may be recommended:

1. Chest X-ray or CT scan: This imaging test can show signs of pneumonia in the lungs.

2. Blood tests: A complete blood count (CBC) and a blood culture test can help identify an infection and indicate the severity of the illness.

3. Sputum culture: This test can identify the specific bacteria causing pneumonia and determine the most effective antibiotic treatment.

4. Influenza test: A nasal or throat swab test can detect the presence of the influenza virus.

5. COVID-19 test: A nasal or throat swab test (PCR test) can confirm if the person has COVID-19.

It is important to consult a healthcare provider for proper evaluation and testing to determine the exact cause of the symptoms and receive appropriate treatment.



**Expected Result**: The system should first identify potential diseases (like COVID-19, pneumonia) and then recommend appropriate lab tests (PCR, chest X-ray, blood tests).


## Complex Parallel Chain for Scientific Research

### Scientific Hypothesis Testing Chain
This chain demonstrates how to create complex workflows that branch and merge information.


In [32]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

# Prompt for generating a hypothesis based on an observation
prompt_observation = ChatPromptTemplate.from_template(
    "Based on the observation: {observation}, what is a possible biological explanation or hypothesis?"
)

# Prompt for suggesting an experiment to test a hypothesis
prompt_hypothesis = ChatPromptTemplate.from_template(
    "What experiment could we perform to test the hypothesis: {hypothesis}, considering the condition: {condition}?"
)

# Prompt for predicting the outcome of an experiment
prompt_experiment = ChatPromptTemplate.from_template(
    "Given the setup: {experiment_setup}, what might be the expected outcome of this experiment?"
)

### Build Multi-Step Research Chain

In [33]:
# Chain to generate a hypothesis from an observation
hypothesis_generator = (
    {"observation": RunnablePassthrough()}
    | prompt_observation
    | model
    | StrOutputParser()
)

# Chain to suggest an experiment based on the hypothesis, merging with additional input
experiment_suggestion = (
    RunnableParallel(
        {
            "hypothesis": hypothesis_generator,
            "condition": RunnablePassthrough(),
        }  # Merging the hypothesis output with a new input 'condition'
    )
    | prompt_hypothesis
    | model
    | StrOutputParser()
)

# Chain to predict the outcome of the suggested experiment
experiment_outcome = (
    {"experiment_setup": experiment_suggestion}
    | prompt_experiment
    | model
    | StrOutputParser()
)

### Visualize Research Chain

In [34]:
experiment_outcome.get_graph().print_ascii()

           +---------------------------------+         
           | Parallel<experiment_setup>Input |         
           +---------------------------------+         
                            *                          
                            *                          
                            *                          
         +-------------------------------------+       
         | Parallel<hypothesis,condition>Input |       
         +-------------------------------------+       
                   **               ***                
                ***                    ***             
              **                          ***          
    +-------------+                          **        
    | Passthrough |                           *        
    +-------------+                           *        
           *                                  *        
           *                                  *        
           *                                  * 

### Test Scientific Research Chain

In [35]:
# Example invocation to test the chain
result = experiment_outcome.invoke(
    {
        "observation": "Pea plants with round seeds produce mostly round seed offspring, even when crossed with wrinkled seeds.",
        "condition": "controlled pollination",
    }
)
print("Experiment Prediction:", result)

Experiment Prediction: The expected outcome of this experiment, if the hypothesis is true, would be that the majority of the offspring plants produced from the controlled cross between pea plants with round seeds and pea plants with wrinkled seeds would have round seeds. This is because in this scenario, round seeds would be the dominant trait and wrinkled seeds would be the recessive trait. 

Therefore, if the hypothesis is correct, the offspring plants would exhibit a phenotypic ratio of approximately 3:1, with three-quarters of the offspring plants having round seeds and one-quarter having wrinkled seeds. This outcome would provide evidence to support the hypothesis that the trait for seed shape is controlled by a single gene with dominant and recessive alleles.


**Expected Result**: The system should generate a hypothesis about dominant/recessive traits, suggest controlled breeding experiments, and predict outcomes based on genetic principles.


## Debate Chain with Argument Analysis

### Creating a Balanced Debate System
This chain demonstrates how to create a system that can analyze arguments from multiple perspectives.

In [36]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

# Initialize the model
model = ChatOpenAI()

# Prompt to generate an initial argument about a controversial life science topic
generate_argument = (
    ChatPromptTemplate.from_template("Generate an argument about: {input}")
    | model
    | StrOutputParser()
    | {"base_argument": RunnablePassthrough()}
)

# Chain to list the positive aspects (pros) of the argument
arguments_for = (
    ChatPromptTemplate.from_template(
        "List the pros or positive aspects of: {base_argument}"
    )
    | model
    | StrOutputParser()
)

# Chain to list the negative aspects (cons) of the argument
arguments_against = (
    ChatPromptTemplate.from_template(
        "List the cons or negative aspects of: {base_argument}"
    )
    | model
    | StrOutputParser()
)

# Template for the final responder to synthesize the debate
final_responder = (
    ChatPromptTemplate.from_template(
        "Discussion on {input}:\n\nPros:\n{arguments_for}\n\nCons:\n{arguments_against}\n\nCan you provide a balanced conclusion?"
    )
    | model
    | StrOutputParser()
)

### Complete Debate Chain

In [37]:
# Full chain to manage the debate
main_chain = (
    generate_argument
    | {
        "arguments_for": arguments_for,
        "arguments_against": arguments_against,
        "input": itemgetter("base_argument"),
    }
    | final_responder
)

### Visualize Debate Chain

In [38]:
main_chain.get_graph().print_ascii()

                                   +-------------+                               
                                   | PromptInput |                               
                                   +-------------+                               
                                          *                                      
                                          *                                      
                                          *                                      
                               +--------------------+                            
                               | ChatPromptTemplate |                            
                               +--------------------+                            
                                          *                                      
                                          *                                      
                                          *                                      
                

### Test Debate Chain

In [39]:
result = main_chain.invoke({"input": "the use of CRISPR technology in human embryos"})
print("Debate Summary:", result)

Debate Summary: In conclusion, the use of CRISPR technology in human embryos has the potential to bring significant benefits in terms of eliminating genetic diseases and disorders, improving quality of life, and advancing medical treatments. However, it is crucial to address the ethical concerns raised by critics, including the potential for designer babies, widening social and economic inequalities, unintended consequences, and the need for responsible and ethical use of this technology.

It is important to carefully consider the implications of using CRISPR technology in embryos and to establish ethical guidelines and regulations to ensure that it is used responsibly and for the greater good of society. Further research and discussion are needed to weigh the potential benefits and risks of this technology and to determine the best way forward in harnessing its potential while mitigating any negative consequences.


**Expected Result**: The system should generate a balanced analysis covering benefits (treating genetic diseases) and concerns (ethical implications, safety) of CRISPR technology.



## Part 7: Custom Chain Functions

### What are Custom Chain Functions?
Custom chain functions allow you to create reusable chain components with the `@chain` decorator, providing more flexibility than standard LCEL chains.



### Drug Discovery Analysis Chain

In [40]:
from langchain_core.runnables import chain

generate_scenario_prompt = ChatPromptTemplate.from_template(
    "Generate a hypothetical drug discovery scenario involving {topic}"
)
analyze_scenario_prompt = ChatPromptTemplate.from_template(
    "What are the key challenges and potential solutions in this scenario: {scenario}"
)


@chain
def drug_discovery_analysis(topic):
    # Generate a scenario based on the given drug discovery topic
    initial_scenario = generate_scenario_prompt.invoke({"topic": topic})
    scenario_description = ChatOpenAI().invoke(initial_scenario)
    parsed_scenario = StrOutputParser().invoke(scenario_description)

    # Analyze the generated scenario to identify challenges and potential solutions
    analysis_chain = analyze_scenario_prompt | ChatOpenAI() | StrOutputParser()
    scenario_analysis = analysis_chain.invoke({"scenario": parsed_scenario})

    return scenario_analysis

In [41]:
topic_focus = "Alzheimer's disease"
scenario_analysis_result = drug_discovery_analysis.invoke(topic_focus)
print("Analysis of Drug Discovery Scenario:", scenario_analysis_result)

Analysis of Drug Discovery Scenario: Key challenges in this scenario:

1. Safety and efficacy: The primary challenge is ensuring the safety and efficacy of the drug candidate in clinical trials. Adverse effects or lack of therapeutic benefit could pose significant hurdles to regulatory approval.

2. Scaling up production: Once the drug candidate shows promising results in clinical trials, scaling up production to meet the demand for a widespread patient population can be a challenge. Ensuring consistent quality and supply of the drug is crucial.

3. Regulatory hurdles: Navigating the regulatory approval process can be complex and time-consuming. Meeting the requirements of regulatory agencies to demonstrate the safety and efficacy of the drug candidate is essential for market approval.

4. Cost of development: Developing a new drug can be expensive, and securing funding for ongoing research and clinical trials can be a challenge. Finding financial resources to support the development p


**Expected Result**: The system should generate a realistic drug discovery scenario for Alzheimer's disease and analyze key challenges like blood-brain barrier penetration, clinical trial design, and regulatory approval.

## Summary

This notebook demonstrated several key concepts in building personal assistants with LangChain:

1. **History-Aware Retrieval**: Building systems that understand conversation context
2. **LCEL Chains**: Using the pipe operator for clean, declarative chain building
3. **Parallel Processing**: Improving efficiency with simultaneous operations
4. **Sequential Chains**: Multi-step reasoning for complex tasks
5. **Complex Workflows**: Branching and merging information streams
6. **Custom Chain Functions**: Creating reusable chain components

These patterns form the foundation for building sophisticated AI assistants that can handle complex, multi-step reasoning tasks while maintaining context and providing accurate, relevant responses.